In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from collections import deque
import pygame
from pygame.locals import *
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
import os
from docx import Document
import win32com.client  # For Word integration on Windows

# Initialize deques to store points
bpoints = [deque(maxlen=1024)]
blue_index = 0
kernel = np.ones((5, 5), np.uint8)
colors = [(0, 0, 0)]
colorIndex = 0

paintWindow = np.zeros((400, 600, 3)) + 255
cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)

mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.5)
mpDraw = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open video capture.")
    exit()

# Flag to track if the document has been opened
first_write = True

def write_to_word(predicted_label, doc_path="recognized_text.docx"):
    global first_write
    try:
        word_app = win32com.client.Dispatch("Word.Application")
        doc_open = False

        for doc in word_app.Documents:
            if doc.FullName == os.path.abspath(doc_path):
                doc_open = True
                break

        if doc_open:
            word_app.Selection.EndKey(6)
            word_app.Selection.TypeText(" " + predicted_label)
            doc.Save()
            print(f"Updated open Word document with: {predicted_label}")
        else:
            if os.path.exists(doc_path):
                doc = Document(doc_path)
                paragraphs = doc.paragraphs
                if paragraphs and paragraphs[-1].text:
                    paragraphs[-1].add_run(" " + predicted_label)
                else:
                    doc.add_paragraph(predicted_label)
            else:
                doc = Document()
                doc.add_heading('Recognized Kannada Characters', level=1)
                doc.add_paragraph(predicted_label)
            doc.save(doc_path)
            print(f"Saved to {doc_path}")

            if first_write:
                word_app.Documents.Open(os.path.abspath(doc_path))
                word_app.Visible = True
                first_write = False
                print(f"Opened {doc_path} in Word")

    except PermissionError:
        temp_doc_path = "recognized_text_temp.docx"
        print(f"Permission denied for {doc_path}. Saving to {temp_doc_path} instead.")
        doc = Document()
        doc.add_paragraph(predicted_label)
        doc.save(temp_doc_path)
    except Exception as e:
        print(f"Error updating document: {e}")

def predict_from_image(image_path):
    pygame.init()
    model_path = r'D:/character_recognition_model_cnn.h5'
    try:
        model = load_model(model_path)
    except Exception as e:
        print(f"Error loading model: {e}")
        pygame.quit()
        return None

    class_indices_inverted = {
        0: 'ಅ', 1: 'ಈ', 2: 'ಇ', 3: 'ಆ', 4: 'ಅಮ್ಮ', 5: 'ನಾನು', 6: 'ಶಾಲೆ',7: 'ಉ', 8: 'ಊ', 9: 'ಋ', 10: 'ಎ', 11: 'ಏ', 12: 'ಐ', 13: 'ಒ',14: 'ಓ', 15: 'ಔ', 16: 'ಅಂ', 17: 'ಅಃ'
    }

    screen_width, screen_height = 400, 400
    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption('Kannada Character Prediction')
    font_path = 'D:/Img/noto/NotoSansKannada_Condensed-Regular.ttf'
    font_size = 100
    font = pygame.font.Font(font_path, font_size)
    text_color = (0, 0, 0)
    background_color = (255, 255, 255)

    try:
        img = load_img(image_path, target_size=(28, 28), color_mode='grayscale')
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions)
        predicted_label = class_indices_inverted.get(predicted_class, 'Unknown')
        print(f"Predicted Label: {predicted_label}")
        write_to_word(predicted_label)

        running = True
        while running:
            for event in pygame.event.get():
                if event.type == QUIT:
                    running = False
            screen.fill(background_color)
            if predicted_label:
                text_surface = font.render(predicted_label, True, text_color)
                text_rect = text_surface.get_rect(center=(screen_width // 2, screen_height // 2))
                screen.blit(text_surface, text_rect)
            pygame.display.flip()
        pygame.quit()
        return predicted_label
    except Exception as e:
        print(f"Error predicting image: {e}")
        pygame.quit()
        return None

def is_thumbs_up(landmarks):
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    middle_tip = landmarks[12]
    ring_tip = landmarks[16]
    pinky_tip = landmarks[20]
    return (thumb_tip[1] < index_tip[1] and thumb_tip[1] < middle_tip[1] and
            thumb_tip[1] < ring_tip[1] and thumb_tip[1] < pinky_tip[1])

ret = True
while ret:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    x, y, c = frame.shape
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    frame = cv2.rectangle(frame, (40, 1), (140, 65), (0, 0, 0), 2)
    cv2.putText(frame, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

    result = hands.process(framergb)
    if result.multi_hand_landmarks:
        landmarks = []
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                lmx = int(lm.x * 640)
                lmy = int(lm.y * 480)
                landmarks.append([lmx, lmy])
            mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

            fore_finger = (landmarks[8][0], landmarks[8][1])
            center = fore_finger
            thumb = (landmarks[4][0], landmarks[4][1])
            cv2.circle(frame, center, 3, (0, 255, 0), -1)

            if thumb[1] - center[1] < 70:
                bpoints.append(deque(maxlen=512))
                blue_index += 1
            elif center[1] <= 65:
                if 40 <= center[0] <= 140:
                    bpoints = [deque(maxlen=512)]
                    blue_index = 0
                    paintWindow[67:, :, :] = 255
            else:
                bpoints[blue_index].appendleft(center)

            # Screenshot capture with guide lines
            if is_thumbs_up(landmarks):
                canvas_image = paintWindow[67:, :, :].copy()
                displayPaint = paintWindow.copy()
                cv2.line(displayPaint, (0, 150), (600, 150), (255, 0, 0), 5)
                cv2.line(displayPaint, (0, 250), (600, 250), (255, 0, 0), 5)
                cv2.line(displayPaint, (0, 350), (600, 350), (0, 0, 0), 2)
                cv2.imwrite('screenshot.png', canvas_image)
                print("Screenshot taken and saved as 'screenshot.png'")
                predict_from_image('screenshot.png')
    else:
        bpoints.append(deque(maxlen=512))
        blue_index += 1

    for i in range(len(bpoints)):
        for k in range(1, len(bpoints[i])):
            if bpoints[i][k - 1] is None or bpoints[i][k] is None:
                continue
            cv2.line(frame, bpoints[i][k - 1], bpoints[i][k], colors[colorIndex], 12)
            cv2.line(paintWindow, bpoints[i][k - 1], bpoints[i][k], colors[colorIndex], 12)

    displayPaint = paintWindow.copy()
    cv2.line(displayPaint, (0, 150), (600, 150), (255, 0, 0), 5)
    cv2.line(displayPaint, (0, 250), (600, 250), (0, 0, 0), 3)
    cv2.line(displayPaint, (0, 350), (600, 350), (255, 0, 0), 5)

    # Also draw lines on the video capture screen
    cv2.line(frame, (0, 150), (600, 150), (255, 0, 0), 5)
    cv2.line(frame, (0, 250), (600, 250), (0, 0, 0), 3)
    cv2.line(frame, (0, 350), (600, 350), (255, 0, 0), 5)

    cv2.imshow("Output", frame)
    cv2.imshow("Paint", displayPaint)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pygame.quit()